In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import os
def fetch_image_dataset(path:str):
    apple = {}
    grape = {}
    for subd in os.listdir(path):
        # inside images
        if subd.startswith('Apple'):
            # apple = {'apple_healthy': [], 'apple_black_rot': [], 'apple_cedar_rust': [], 'apple_scab': []}
            apple[subd] = []
            for img in os.listdir(os.path.join(path,subd)):
                apple[subd].append(img)
        elif subd.startswith('Grape'):
            # grape = {'grape_healthy': [], 'grape_black_rot': [], 'apple_esca': [], 'apple_spot': []}
            grape[subd] = []
            for img in os.listdir(os.path.join(path,subd)):
                grape[subd].append(img)
        else:
            print('Plant not supported')
    return apple, grape

In [4]:
apple, grape = fetch_image_dataset('C:/Users/khali/Desktop/Leaffliction/images')

In [5]:
def analyze_distribution(data:dict):
    dist = {}
    for k,v in data.items():
        dist[k] = len(v)
    return dist

In [6]:
apple_dist = analyze_distribution(apple)
grape_dist = analyze_distribution(grape)

In [13]:

def plot_distribution(data: dict):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Define color palette
    colors = plt.cm.tab10.colors
    
    # Bar chart
    ax1.bar(data.keys(), data.values(), color=colors[:len(data)])
    # Pie chart
    ax2.pie(data.values(), labels=data.keys(), autopct='%1.1f%%', colors=colors[:len(data)])
    
    plt.title(f'{list(data.keys())[0][:5]} Distribution')
    plt.tight_layout()
    plt.show()


In [ ]:
plot_distribution(apple_dist)
plot_distribution(grape_dist)

In [33]:
import torch
from torchvision.transforms import v2
from PIL import Image
import os
import random

def Augmentation(path:str):
    for subd in os.listdir(path):
        newpath = os.path.join(path, subd)
        os.makedirs(newpath + '/Augmented', exist_ok=True)
        for img in os.listdir(newpath):
            if img.lower().endswith('.jpg'):
                image = Image.open(os.path.join(newpath +'/', img))

                blur = v2.GaussianBlur(kernel_size=(3, 7), sigma=(1.5, 6))(image)
                blur.save(os.path.join(newpath + '/Augmented', img.split('.')[0] + "_blur.jpg"))

                rotate = v2.RandomRotation(degrees=[-80, 80])(image)
                rotate.save(os.path.join(newpath + '/Augmented', img.split('.')[0] + "_rotate.jpg"))
                
                contrast = v2.ColorJitter(brightness=(0.5, 2), contrast=(0.5, 2), saturation=(0.5, 2))(image)
                contrast.save(os.path.join(newpath + '/Augmented', img.split('.')[0] + "_contrast.jpg"))
                
                scaling = v2.RandomResizedCrop(size=(200, 200))(image)
                scaling.save(os.path.join(newpath + '/Augmented', img.split('.')[0] + "_scaling.jpg"))
                
                shear = v2.RandomAffine(degrees=0, shear=45)(image)
                shear.save(os.path.join(newpath + '/Augmented', img.split('.')[0] + '_shear.jpg'))
                
                if random.randint(0, 1) == 0:
                    flip = v2.functional.hflip(image)
                else:
                    flip = v2.functional.vflip(image)
                flip.save(os.path.join(newpath + '/Augmented', img.split('.')[0] + '_flip.jpg'))

In [34]:
path = "C:\\Users\\khali\\Desktop\\LEAFFLICTION\\images"
Augmentation(path)

In [28]:
from plantcv import plantcv as pcv
import matplotlib.pyplot as plt
import os

def gaussian_blur(mask):
    gaussian_img = pcv.gaussian_blur(img=mask, ksize=(3, 3))

def apply_mask(image, mask):
    masked = pcv.apply_mask(img=image, mask=mask, mask_color='white')

def rect_roi(image):
    rect_roi = pcv.roi.rectangle(img=image, x=35, y=8, h=235, w=200)

def shape_analysis(image, mask):
    shape_img = pcv.analyze.size(img=image, labeled_mask=mask)

def x_pseudolandmarks(image, mask):
    top, bottom, center_v = pcv.homology.x_axis_pseudolandmarks(img=image, mask=mask)

def y_pseudolandmarks(image, mask):
    left, right, center_h = pcv.homology.y_axis_pseudolandmarks(img=image, mask=mask)

def Transformation(src:str, dest:str = None):
    if src.lower().endswith('.jpg'):
        pcv.params.debug = "plot"
        image, _, _ = pcv.readimage(filename=src)

        # Identify the grayscale image that maximizes the gray value difference
        # between the plant and the background with:
        # cs = pcv.visualize.colorspaces(rgb_img=image, original_img=False)
        # We deduce that the 'LAB' colorspace's channel 'a' is the best option 
        # for our grayscale image
        grayscale_img = pcv.rgb2gray_lab(rgb_img=image, channel="a")
        
        # Identify plant from background, by converting the grayscale image 
        # to a binary image (MASK) using a threshold value of '119' which is 
        # the minimum value between the two peaks in our histogram:
        # hist = pcv.visualize.histogram(img=grayscale_img, bins=30)
        s_thresh = pcv.threshold.binary(grayscale_img, threshold=119, object_type="dark")
        # Apply Transformations
        gaussian_blur(s_thresh)
        apply_mask(image, s_thresh)
        rect_roi(image)
        shape_analysis(image, s_thresh)
        x_pseudolandmarks(image, s_thresh)
        y_pseudolandmarks(image, s_thresh)
    else:
        pcv.params.debug = "print"
        if dest:
            pcv.params.debug_outdir = dest
            os.makedirs(f"{dest}", exist_ok=True)
        else:
            pcv.params.debug_outdir = f"{src}/Transformed"
            os.makedirs(f"{src}/Transformed", exist_ok=True)
        for img in os.listdir(src):
            if img.lower().endswith('.jpg'):
                image, _, _ = pcv.readimage(filename=os.path.join(src, img))
                grayscale_img = pcv.rgb2gray_lab(rgb_img=image, channel="a")
                s_thresh = pcv.threshold.binary(grayscale_img, threshold=119, object_type="dark")
                gaussian_blur(s_thresh)
                apply_mask(image, s_thresh)
                rect_roi(image)
                shape_analysis(image, s_thresh)
                x_pseudolandmarks(image, s_thresh)
                y_pseudolandmarks(image, s_thresh)
    
    pcv.outputs.save_results(filename=f"result.json", outformat="json")

In [ ]:
src = "C:\\Users\\khali\\Desktop\\LEAFFLICTION\\images\\Apple_healthy\\image (1).jpg"
Transformation(src)